# Quick start

Let's quickly illustrate how to work with the `udlai` and what to expect as an outcome.

The following notebook illustrates how to call the UDL Feature API using the library and describes each of the steps to follow.

In [1]:
import udlai

Before we can do anything, we need an authentication token. See [udl.ai](https://udl.ai) for more details on how to get one.

In [2]:
token = '2854d7073ad847f69ab0bf576132e064726b8b77' # this is just an illustration, not a working token

## Attributes

Let's start with an overview of available attributes. We just need to pass the token.

In [3]:
attributes = udlai.attributes(token)

Now we have a pandas DataFrame containing all attributes we have an access to.

In [4]:
attributes

,id,name,description,short_description,unit,tags.id,tags.name,main_tag.id,main_tag.name,data_version,...,source_provider,source_provider_link,coverage_general,epsg_code,min_value,max_value,standard_deviation,mean,year,value_formatter
0,99,Area accessible 2min driving,Catchment area accessible in 2 min freeflow dr...,None,m²,105,Morphology,105,Morphology,2016,...,Swiss Topo TLM,https://www.swisstopo.admin.ch/de/wissen-fakte...,Switzerland,2056.0,8.0,25263465.0,834619.117500,488624.420100,2016,None
1,242,Usefit for residential,Potential for residential usage. The predictio...,None,prob,113,Real Estate Market,113,Real Estate Market,2016,...,UDL,None,None,2056.0,0.0,1.0,0.300000,0.830000,2016,None
2,287,Pre-school distance,Air distance to next pre-school.,None,m,111,Points of Interest,111,Points of Interest,2016,...,BFS,https://www.bfs.admin.ch/bfs/de/home/statistik...,None,2056.0,0.0,34605.0,3520.000000,3239.000000,2016,None
3,293,Distance primary school,Distance to next primary school.,None,m,111,Points of Interest,111,Points of Interest,2016,...,BFS,https://www.bfs.admin.ch/bfs/de/home/statistik...,None,2056.0,0.0,20000.0,370.000000,700.000000,2016,None
4,140,Slope of Terrain,Slope of terrain.,None,degree,106,Topography,106,Topography,2016,...,Swiss Topo TLM,https://www.swisstopo.admin.ch/de/geodata/heig...,None,2056.0,0.0,35.0,4.383386,4.600148,2016,None
5,172,Leisure activties in 300m,Leisure activities in 300m.,None,-,111,Points of Interest,111,Points of Interest,2016,...,BFS,https://www.bfs.admin.ch/bfs/de/home/statistik...,None,2056.0,0.0,133.0,3.908146,0.826799,2016,None
6,166,Public transport distance,Air distance to next public transport stop.,None,m,111,Points of Interest,111,Points of Interest,2017,...,SBB,https://opentransportdata.swiss/dataset/bhlist,None,2056.0,0.0,2750.0,397.933136,530.353548,2017,None
7,288,Distance secondary school,Distance secondary school,None,[m],111,Points of Interest,111,Points of Interest,None,...,None,None,None,NaN,0.0,37884.0,3822.000000,4044.000000,None,None
8,289,Usefit Commercial,Usefit Commercial,None,prob,113,Real Estate Market,113,Real Estate Market,None,...,None,None,None,NaN,0.0,1.0,0.300000,0.100000,None,None
9,169,Retail store distance,Air distance to next retail store.,None,m,111,Points of Interest,111,Points of Interest,2016,...,BFS,https://www.bfs.admin.ch/bfs/de/home/statistik...,None,2056.0,0.0,4720.0,648.461963,1024.748356,2016,None


We can also query detials of a single attribute, if we know its ID.

In [5]:
attribute_298 = udlai.attribute_detail(token, attribute_id=289)

The result is a pandas Series.

In [6]:
attribute_289

id                                     289
name                     Usefit Commercial
description              Usefit Commercial
short_description                     None
unit                                  prob
tags.id                                113
tags.name               Real Estate Market
main_tag.id                            113
main_tag.name           Real Estate Market
data_version                          None
data_last_update                      None
data_processor                        None
source_provider                       None
source_provider_link                  None
coverage_general                      None
epsg_code                             None
min_value                              0.0
max_value                              1.0
standard_deviation                     0.3
mean                                   0.1
year                                  None
value_formatter                       None
dtype: object

## Features

So far, we have not accessed any actual values, just the metadata. But given spatial coordinates (latitude and longitude), we can query UDL.AI platform and retrieve the values of selected attributes for any place in the database. We can query a single attribute or multiple at once. The same applies to coordinates.

Single point and a sigle attribute:

In [9]:
single_point_single_attr = udlai.features(token, latitude=47.37, longitude=8.54, attribute_id=99)
single_point_single_attr

99    1114508
Name: (47.37, 8.54), dtype: object

 Single point and multiple attributes, indexed by name instead on an attribute ID:

In [13]:
single_point_name = udlai.features(token, latitude=47.37, longitude=8.54, attribute_id=[289, 99], index_by="id")
single_point_name

289    0.922094166278839
99               1114508
Name: (47.37, 8.54), dtype: object

Multiple points and a single attribute (the last point is not in the existing database):

In [14]:
lats = [47.3769267, 47.3769267, 48.3769267]
lons = [8.5497381, 8.5417981, 8.9417981]
multiple_pts = udlai.features(token, latitude=lats, longitude=lons, attribute_id=[289, 99])
multiple_pts

/Users/martin/Git/udlai/udlai/feature_api.py:338: UserWarning: Some of the locations are not within the udl.ai database. Have you passed correct coordinates?
  warnings.warn(


,latitude,longitude,289,99
0,47.376927,8.549738,0.950480580329895,801361
1,47.376927,8.541798,0.917928218841553,606369
2,48.376927,8.941798,None,None


Multiple points and a multiple attributes, again indexed by name:

In [16]:
lats = [47.3769267, 47.3769267, 48.3769267]
lons = [8.5497381, 8.5417981, 8.9417981]
ids = [99, 242, 171, 289]
multiple_data = udlai.features(token, latitude=lats, longitude=lons, attribute_id=ids, index_by="name")
multiple_data

/Users/martin/Git/udlai/udlai/feature_api.py:338: UserWarning: Some of the locations are not within the udl.ai database. Have you passed correct coordinates?
  warnings.warn(


,latitude,longitude,net_prox_size,usefit_resdnt,retail_300,usefit_com
0,47.376927,8.549738,801361,0.0296293534338474,0,0.950480580329895
1,47.376927,8.541798,606369,0.0284973289817572,5,0.917928218841553
2,48.376927,8.941798,None,None,None,None


You can query as many coordinates and as many attributes as your token allows.

## Summary statistics over an area

You can also query a summary statistics over a given area. That can be passed as a `shapely.geometry` Polygon or MultiPolygon in EPSG:4326 or as a GeoJSON representation.

In [24]:
geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [8.5367, 47.3712],
            [8.5406, 47.3712],
            [8.5406, 47.3739],
            [8.5367, 47.3739],
            [8.5367, 47.3712],
        ]
    ],
}
agg = udlai.aggregates(token, geojson, 242, index_by="name")
agg

,max,mean,median,min,std,sum
Usefit for residential,0.977297,0.365754,0.238565,0.008558,0.351673,53.034321
